## Regresyon Algoritmalarına Bakış

In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import load_boston
boston = load_boston()

bos = pd.DataFrame(boston.data)
bos.columns = boston.feature_names
print(bos.head())

In [ ]:
bos['PRICE'] = boston.target
X = bos.drop('PRICE', axis = 1)
Y = bos['PRICE']

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
x_train, x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=0)

In [ ]:
sc=StandardScaler()
sc.fit(x_train)
x_train_std=sc.transform(x_train)
x_test_std=sc.transform(x_test)

## 1. Doğrusal Regresyon

In [ ]:
from sklearn.linear_model import LinearRegression

linreg = LinearRegression().fit(x_train_std, y_train)

print('doğrusal model sabit: {}'     .format(linreg.intercept_))
print('doğrusal model katsayılar (w): {}'     .format(linreg.coef_))
print('R-kare (eğitim): {:.3f}'     .format(linreg.score(x_train_std, y_train.values.ravel())))
print('R-kare (test): {:.3f}'     .format(linreg.score(x_test_std, y_test.values.ravel())))

## 2. Ridge Regresyon

In [ ]:
from sklearn.linear_model import Ridge

linridge = Ridge().fit(x_train_std, y_train)

print('ridge regresyon sabit: {}'     .format(linridge.intercept_))
print('ridge regresyon katsayılar:\n{}'     .format(linridge.coef_))
print('R-kare (eğitim): {:.3f}'     .format(linridge.score(x_train_std, y_train.values.ravel())))
print('R-kare (test): {:.3f}'     .format(linridge.score(x_test_std, y_test.values.ravel())))
print('0 olmayan öznitelik sayısı: {}'     .format(np.sum(linridge.coef_ != 0)))

## 3. Lasso Regresyon

In [ ]:
from sklearn.linear_model import Lasso

linlasso = Lasso(max_iter = 10000).fit(x_train_std, y_train)

print('lasso regresyon sabit: {}'     .format(linlasso.intercept_))
print('lasso regresyon ağırlıklar:\n{}'     .format(linlasso.coef_))
print('0 olmayan öznitelikler: {}'     .format(np.sum(linlasso.coef_ != 0)))
print('R-kare (eğitim): {:.3f}'     .format(linlasso.score(x_train_std, y_train.values.ravel())))
print('R-kare (test): {:.3f}\n'     .format(linlasso.score(x_test_std, y_test.values.ravel())))
print('0 olmayan özniteliklerin ağırlıkları (mutlak değer sırasında):')

for e in sorted (list(zip(list(X), linlasso.coef_)),
                key = lambda e: -abs(e[1])):
    if e[1] != 0:
        print('\t{}, {:.3f}'.format(e[0], e[1]))

## 4. Doğrusal Olmayan Regresyon (Polynominal Regression)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=5)
X_poly = poly.fit_transform(X)

X_train, X_test, y_train1, y_test1 = train_test_split(X_poly, Y,random_state = 0)
linreg1 = LinearRegression().fit(X_train, y_train)

#print('(polinom model) katsayılar (w):\n{}'     .format(linreg.coef_))
print('(polinom model) sabit (b): {:.3f}'     .format(linreg1.intercept_))
print('(polinom model) R-kare (eğitim): {:.3f}'     .format(linreg1.score(X_train, y_train1.values.ravel())))
print('(polinom model) R-kare (test): {:.3f}\n'     .format(linreg1.score(X_test, y_test1.values.ravel())))
print('0 olmayan öznitelik sayısı: {}'     .format(np.sum(linreg1.coef_ != 0)))

pre_train=linreg1.predict(X_train)
pre_test= linreg1.predict(X_test)
print("MSE-eğitim: %.3f"      % mean_squared_error(y_train1, pre_train))
print("MSE-test: %.3f\n"      % mean_squared_error(y_test1, pre_test))

print('R-Kare (eğitim): %.3f' % r2_score(y_train1, pre_train))
print('R-Kare (test): %.3f' % r2_score(y_test1, pre_test))

## 5. Doğrusal Olmayan Regresyon (Normalleştirme ile)

In [ ]:
poly = PolynomialFeatures(degree=5)
X_poly = poly.fit_transform(X)
X_train, X_test, y_train1, y_test1 = train_test_split(X_poly, Y,random_state = 0)

sc.fit(X_train)
X_train_std=sc.transform(X_train)
X_test_std=sc.transform(X_test)
linreg1 = LinearRegression().fit(X_train_std, y_train1)

#print('(polinom model) katsayılar (w):\n{}'     .format(linreg.coef_))
print('(polinom model) sabit (b): {:.3f}'     .format(linreg1.intercept_))
print('(polinom model) R-kare (eğitim): {:.3f}'     .format(linreg1.score(X_train_std, y_train1.values.ravel())))
print('(polinom model) R-kare (test): {:.3f}\n'     .format(linreg1.score(X_test_std, y_test1.values.ravel())))
print('0 olmayan öznitelik sayısı: {}'     .format(np.sum(linreg1.coef_ != 0)))

pre_train=linreg1.predict(X_train_std)
pre_test= linreg1.predict(X_test_std)
print("MSE-eğitim: %.3f"      % mean_squared_error(y_train1, pre_train))
print("MSE-test: %.3f\n"      % mean_squared_error(y_test1, pre_test))


## 6. K-nn Regresyon 

In [ ]:
from sklearn.neighbors import KNeighborsRegressor


kNN = KNeighborsRegressor(n_neighbors=1)
kNN.fit(x_train_std, y_train)

pred_train=kNN.predict(x_train_std)
pred_test= kNN.predict(x_test_std)
print('R-kare (eğitim): {:.3f}'     .format(kNN.score(x_train_std, y_train.values.ravel())))
print('R-kare (test): {:.3f}'     .format(kNN.score(x_test_std, y_test.values.ravel())))
print("MSE-eğitim: %.3f"      % mean_squared_error(y_train, pred_train))
print("MSE-test: %.3f\n"      % mean_squared_error(y_test, pred_test))

## 7. Karar Ağacı Regresyonu

In [ ]:
from sklearn.tree import DecisionTreeRegressor

KAR = DecisionTreeRegressor(max_leaf_nodes=20, random_state=0)
KAR.fit(x_train, y_train)

pred_train=KAR.predict(x_train)
pred_test= KAR.predict(x_test)
print('R-kare (eğitim): {:.3f}'     .format(KAR.score(x_train, y_train)))
print('R-kare (test): {:.3f}'     .format(KAR.score(x_test, y_test)))
print("MSE-eğitim: %.3f"      % mean_squared_error(y_train, pred_train))
print("MSE-test: %.3f"      % mean_squared_error(y_test, pred_test))

## 8. Yapay Sinir Ağları

In [ ]:
from sklearn.neural_network import MLPRegressor

# activation : {‘identity’, ‘logistic’, ‘tanh’, ‘relu’}
# solver: adam (büyük data varsa tercih), lbfgs
mlp = MLPRegressor(activation='relu',hidden_layer_sizes=[10],solver='lbfgs',max_iter=200000,tol=0.000000001)
mlp.fit(x_train_std,y_train.values.ravel())


In [ ]:
pre_train=mlp.predict(x_train_std)
pre_test= mlp.predict(x_test_std)

print('R-kare (eğitim): %.4f' % r2_score(y_train, pre_train))   
print('R-kare (test): %.4f\n' % r2_score(y_test, pre_test))
print("MSE-Eğitim: %.3f"      % mean_squared_error(y_train, pre_train))
print("MSE-Test: %.3f"      % mean_squared_error(y_test, pre_test))

## 9. XGBoost

In [ ]:
import xgboost 
from sklearn.metrics import explained_variance_score

xgb = xgboost.XGBRegressor(objective ='reg:linear',n_estimators=200, learning_rate=0.08, max_depth=10)
xgb.fit(x_train,y_train)

In [ ]:
pre_train=xgb.predict(x_train)
pre_test= xgb.predict(x_test)

print('R-kare (eğitim): %.4f' % r2_score(y_train, pre_train))   
print('R-kare (test): %.4f\n' % r2_score(y_test, pre_test))
print("MSE-Eğitim: %.3f"      % mean_squared_error(y_train, pre_train))
print("MSE-Test: %.3f"      % mean_squared_error(y_test, pre_test))

## 10. Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_clf = RandomForestRegressor(max_depth=15,n_estimators=100, min_samples_leaf=10,min_samples_split=100, random_state=10)
rf_clf.fit(x_train,y_train)

In [ ]:
pre_train=rf_clf.predict(x_train)
pre_test= rf_clf.predict(x_test)

print('R-kare (eğitim): %.4f' % r2_score(y_train, pre_train))   
print('R-kare (test): %.4f\n' % r2_score(y_test, pre_test))
print("MSE-Eğitim: %.3f"      % mean_squared_error(y_train, pre_train))
print("MSE-Test: %.3f"      % mean_squared_error(y_test, pre_test))

# PARAMETRE OPTİMİZASYONU ZAMANI

## 1. Ridge regresyon ve düzenlileştirme katsayısı

In [ ]:
print('Ridge regresyon: düzenlileştirme katsayısının etkisi\n')
for alfa in [0.05, 0.1, 0, 1, 10, 20, 50, 100, 1000]:
    linridge = Ridge(alpha = alfa).fit(x_train_std, y_train)
    print("lambda= {:.2f}\n ".format(alfa))
    pre_train=linridge.predict(x_train_std)
    pre_test= linridge.predict(x_test_std)
    print('R-Kare (eğitim): %.3f' % r2_score(y_train, pre_train))
    print('R-Kare (test): %.3f\n' % r2_score(y_test, pre_test))

## 2. Lasso regresyon ve düzenlileştirme katsayısı

In [ ]:
print('Lasso regresyon: düzenlileştirme katsayısının etkisi\n')
for alfa in [0.001, 0,.01, 0.5, 1, 2, 3, 5, 10, 20, 50]:
    linlasso = Lasso(alpha = alfa,max_iter = 150000).fit(x_train_std, y_train)
    print("lambda= {:.2f}\n ".format(alfa))
    pre_train=linlasso.predict(x_train_std)
    pre_test= linlasso.predict(x_test_std)
    print('R-Kare (eğitim): %.3f' % r2_score(y_train, pre_train))
    print('R-Kare (test): %.3f\n' % r2_score(y_test, pre_test))
    print('0 olmayan öznitelikler: {}\n'     .format(np.sum(linlasso.coef_ != 0)))

## 3. Polinom Derecesi

In [ ]:
for a in range(1,7):
    tic = time.time()
    tahminpolinom=PolynomialFeatures(degree=a)
    Xyeni=tahminpolinom.fit_transform(X)
    X_train1, X_test1, y_train1, y_test1 = train_test_split(Xyeni, Y,test_size=0.3,random_state=0)
    sc=StandardScaler()
    sc.fit(X_train1)
    x_train_st1=sc.transform(X_train1)
    x_test_st1=sc.transform(X_test1)
    
    polimodel=LinearRegression()
    polimodel.fit(x_train_st1, y_train1)
    pre_train=polimodel.predict(x_train_st1)
    pre_test= polimodel.predict(x_test_st1)
    print("derece",a)
    print('\n')
    print("MSE-eğitim: %.3f"      % mean_squared_error(y_train1, pre_train))
    print("MSE-test: %.3f\n"      % mean_squared_error(y_test1, pre_test))
    print('R-Kare (eğitim): %.3f' % r2_score(y_train1, pre_train))
    print('R-Kare (test): %.3f\n' % r2_score(y_test1, pre_test))
    print('0 olmayan öznitelik sayısı: {}'     .format(np.sum(polimodel.coef_ != 0)))
    toc = time.time()
    print (toc-tic, "sn geçti\n")